<a href="https://colab.research.google.com/github/FarshadAmiri/Structural_Engineering/blob/master/HPO_1_Mahdi_Structures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
import time

In [164]:
# Dataset = pd.read_excel(r'https://github.com/FarshadAmiri/Structrual_Design_Optimization/blob/master/Dataset%20(213%20samples).xlsx?raw=true')
Dataset = pd.read_excel(r'https://github.com/FarshadAmiri/Structural_Engineering/blob/master/D600.xlsx?raw=true')

# Dataset = pd.read_excel(r'https://github.com/FarshadAmiri/Structural_Engineering/blob/master/D608-m.xlsx?raw=true')
cols = ['Mu33','Mu22','Pu','L','Cb','Delta']

for col in cols:
    col_zscore = col + '_zscore'
    Dataset[col_zscore] = (Dataset[col] - Dataset[col].mean())/Dataset[col].std(ddof=0)

In [165]:
#Defining Trainig data and Test Data
shuffled_dataset = Dataset.reindex(np.random.permutation(Dataset.index))

train_df = shuffled_dataset[:530]
test_df  = shuffled_dataset[530:]

In [166]:
#  assigning features and targets

# x = np.column_stack((train_df.Mu33.values , train_df.Mu22.values, train_df.Pu.values, train_df.L.values, train_df.Cb.values, train_df.Delta.values, train_df.L1tb.values))
# y = np.column_stack((train_df.h.values, train_df.b.values, train_df.tf.values, train_df.tw.values))

# test_x = np.column_stack((test_df.Mu33.values , test_df.Mu22.values, test_df.Pu.values, test_df.L.values, test_df.Cb.values, test_df.Delta.values, test_df.L1tb.values))
# test_y = np.column_stack((test_df.h.values, test_df.b.values, test_df.tf.values, test_df.tw.values))

#Trying 'Z score'
x = np.column_stack((train_df.Mu33_zscore.values , train_df.Mu22_zscore.values, train_df.Pu_zscore.values, train_df.L_zscore.values, train_df.Cb_zscore.values, train_df.Delta_zscore.values, train_df.L1tb.values))
y = np.column_stack((train_df.h.values, train_df.b.values, train_df.tf.values, train_df.tw.values))

test_x = np.column_stack((test_df.Mu33_zscore.values , test_df.Mu22_zscore.values, test_df.Pu_zscore.values, test_df.L_zscore.values, test_df.Cb_zscore.values, test_df.Delta_zscore.values,  test_df.L1tb.values))
test_y = np.column_stack((test_df.h.values, test_df.b.values, test_df.tf.values, test_df.tw.values))

In [167]:
# Function to create model; used by KerasRegressor
def create_model(Lambda,LR,neurons):
  Lambda = 0.001
  LR = 0.002
  neurons = 128
	# create model
  model = keras.Sequential([
    keras.layers.Dense(neurons,input_shape=(7,),activation='relu',kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(neurons,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(neurons,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(neurons,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(neurons,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(neurons,activation='relu', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda)),
    keras.layers.Dense(4, activation='linear', kernel_regularizer= tf.keras.regularizers.L2(l2=Lambda))])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),loss = 'mse',
                metrics = tf.keras.metrics.RootMeanSquaredError(name='rmse', dtype=None))
  return model

In [168]:
# Create model
estimator = tf.keras.wrappers.scikit_learn.KerasRegressor(build_fn=create_model
                                                           , batch_size = 4, epochs=50
                                                      #  , neurons , Lmabda , LR
                                                          )
# KerasRegressor
# KerasClassifier

# Define the grid serach parameters
# neurons = [128,256,512]
# Lambda = [0.001,0.005,0.01,0.015]
# LR = [0.001,0.0015,0.002]
# batch_size = [6,8,10]

neurons = [128,256]
Lambda = [0.001,0.01]
LR = [0.001,0.002]

# batch_size = [4,6,8]
# epochs = [50, 80]
param_grid = dict(LR = LR , neurons = neurons , Lambda = Lambda)

In [169]:
param_grid

{'LR': [0.001, 0.002], 'Lambda': [0.001, 0.01], 'neurons': [128, 256]}

In [170]:
grid = GridSearchCV(estimator=estimator, param_grid= param_grid , n_jobs=-1
                    ,cv=5
                    )
grid_result = grid.fit(x,y)
                      #  ,batch_size=6
                      #  , epochs =50 
                      #  ,validation_split=0.4
                      #  )

Epoch 1/50
133/133 [==============================] - 1s 2ms/step - loss: 595.0410 - rmse: 23.5466
Epoch 2/50
133/133 [==============================] - 0s 3ms/step - loss: 149.3234 - rmse: 12.1850
Epoch 3/50
133/133 [==============================] - 0s 2ms/step - loss: 126.4424 - rmse: 11.2191
Epoch 4/50
133/133 [==============================] - 0s 2ms/step - loss: 120.2185 - rmse: 10.9305
Epoch 5/50
133/133 [==============================] - 0s 2ms/step - loss: 87.0746 - rmse: 9.3016
Epoch 6/50
133/133 [==============================] - 0s 2ms/step - loss: 80.1370 - rmse: 8.8985
Epoch 7/50
133/133 [==============================] - 0s 2ms/step - loss: 70.4882 - rmse: 8.3214
Epoch 8/50
133/133 [==============================] - 0s 2ms/step - loss: 59.9736 - rmse: 7.7022
Epoch 9/50
133/133 [==============================] - 0s 2ms/step - loss: 88.5973 - rmse: 9.3744
Epoch 10/50
133/133 [==============================] - 0s 2ms/step - loss: 38.0624 - rmse: 6.1147
Epoch 11/50
133/133 [

In [171]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: -38.389209 using {'LR': 0.001, 'Lambda': 0.01, 'neurons': 256}
-38.547319 (6.853880) with: {'LR': 0.001, 'Lambda': 0.001, 'neurons': 128}
-43.695145 (11.374176) with: {'LR': 0.001, 'Lambda': 0.001, 'neurons': 256}
-42.411367 (11.691872) with: {'LR': 0.001, 'Lambda': 0.01, 'neurons': 128}
-38.389209 (5.507264) with: {'LR': 0.001, 'Lambda': 0.01, 'neurons': 256}
-48.101289 (14.864439) with: {'LR': 0.002, 'Lambda': 0.001, 'neurons': 128}
-47.458305 (8.563178) with: {'LR': 0.002, 'Lambda': 0.001, 'neurons': 256}
-39.730132 (4.924150) with: {'LR': 0.002, 'Lambda': 0.01, 'neurons': 128}
-42.283516 (9.430414) with: {'LR': 0.002, 'Lambda': 0.01, 'neurons': 256}
